# Articles RecSys

---

В данном ноутбуке содержится мое решение для соревнования по ИАД Articles RecSys. После рассмотрения всех возможных вариантов (после долгого, очень долгого пути проб и ошибок...)  я решила использовать Гибридный подход в данной задаче и применить факторизационную машину (библиотека Lightfm), которая эффективно использует данные о взаимодействиях пользователей с айтемами, а также учитывает метаинформацию о пользователях/айтемах (в нашем случае об айтемах). Таким образом, удается эффективнее справляться с проблемой холодного старта.

In [0]:
! pip3 install lightfm

In [0]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import json
import gc
import time

#import tqdm
from scipy.sparse import csr_matrix,  coo_matrix, vstack, hstack
from scipy.sparse import eye as eye
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer as tfdf
from lightfm import LightFM
from multiprocessing import cpu_count

import string
import re


In [0]:
# код для загрузки датасета с помощью kaggle api (нужен файл kaggle.json)

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json
import kaggle
!kaggle competitions download -c recsys-iad-challenge

! unzip "/content/items.json.zip"
! unzip "/content/train.json.zip"
! unzip "/content/random_benchmark.csv.zip"

  0% 0.00/10.9M [00:00<?, ?B/s]
100% 10.9M/10.9M [00:00<00:00, 100MB/s]
 98% 230M/235M [00:02<00:00, 111MB/s] 
100% 235M/235M [00:02<00:00, 106MB/s]
 96% 353M/368M [00:06<00:00, 28.3MB/s]
100% 368M/368M [00:06<00:00, 56.1MB/s]
Archive:  /content/items.json.zip
  inflating: items.json              
Archive:  /content/train.json.zip
  inflating: train.json              
Archive:  /content/random_benchmark.csv.zip
  inflating: random_benchmark.csv    


In [0]:
# данный код для загрузки описания айтемов был любезно предоставлен нашим преподавателем

%%time

import pandas as pd
import json
from tqdm.auto import tqdm

items_list=[]
with tqdm(open('./items.json')) as inf:
  for line in inf:
    item=json.loads(line)
    items_list.append(item)

items_df=pd.DataFrame(items_list).set_index('itemId')


CPU times: user 13.6 s, sys: 1.66 s, total: 15.3 s
Wall time: 14.8 s


## Пару слов о фичах

---

Я решила использовать  tf-idf как самый доступный и при этом достаточно эффективный метод для получения векторных представлений. Были использованы  title и content, предварительно обработанные с целью удаления знаков препинания. Полученные матрицы, несущие в себе информацию о контенте статьи, были сконкатенированы и скормлены фактаризационной машине в качестве item_features. 

In [0]:
# функция для простенькой предобработки текста
def delete_punctuation(x):


    punctuation = list(string.punctuation)
    return ''.join([a if a not in punctuation + ['«»\n--'] else ' ' for a in x])


items_df.title = items_df.title.apply(delete_punctuation)
items_df.content = items_df.content.apply(delete_punctuation)

In [0]:
# создание фичей
vectorizer_t = tfdf(lowercase=False, min_df=90, max_df=0.01)
tf_idf_titles = vectorizer_t.fit_transform(items_df.title.values)
vectorizer_c = tfdf(lowercase=False, min_df=90, max_df=0.01)
tf_idf_content = vectorizer_c.fit_transform(items_df.content.values)

In [0]:
# identity матрица 
num_items= len(items_df)
a = eye(num_items)
# полученные фичи, которые мы подадим в факторизационную машину
items_features =hstack([a, tf_idf_titles, tf_idf_content], format="csr")
items_features

<328050x387044 sparse matrix of type '<class 'numpy.float64'>'
	with 23797064 stored elements in Compressed Sparse Row format>

In [0]:
# код для загрузки основного датасета и создания из него спарс матрицы 
users = []
items = []
data = []

with open("/content/train.json", 'r') as f:
  for line in f:
    user = json.loads(line)
    for item, rating in user['trainRatings'].items():
      users.append(user['userId'])
      items.append(item)
      if rating == 0:
        data.append(-1)
      else:
        data.append(rating)
  
# основные данные в формате coo_matrix: клики обозначены 1 (позитивное взаимодействие)
# и отсутствие кликов -1 (негативное взаимодействие)
interaction_matrix = coo_matrix((data, (users, list(map(int, items)))))
interaction_matrix

<42977x328050 sparse matrix of type '<class 'numpy.int64'>'
	with 67780168 stored elements in COOrdinate format>

# Обучение модели

---

Итак, в итоге лучший результат был получен с помощью модели с векторами размерностью 150 (были попробованы и другие: но меньше - ухудшается качество, больше - слишком долгое время обучения). В качестве оптимизируемой функции был выбран logloss, так как присутствуют как позитивные, так и негативные взаимодействия. Модель обучалась  5 эпох.

In [0]:
n_Components = 150
n_Epochs = 5
seed= 10

model= LightFM(loss="logistic", no_components=n_Components, random_state=seed)
                          
model.fit_partial(interaction_matrix, item_features=items_features, epochs=n_Epochs, num_threads=cpu_count(), verbose=True)


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


## Создание итогового сабмита для кэггл

---

В этой части все просто - с помощью полученной модели и фичей создаем предсказание для пользователей из файла random_benchmark


In [0]:
prediction = pd.read_csv('/content/random_benchmark.csv')
prediction['preds'] = model.predict(prediction.userId.values,
                                    prediction.itemId.values,
                                    item_features = items_features,
                                    num_threads=cpu_count())


prediction.sort_values(['userId', 'preds'], ascending=[True, False], inplace=True)
prediction.drop('preds', axis=1, inplace=True)

prediction.to_csv('with_identity1.csv', index=False)